In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

In [2]:
NAME = "Cats-vs-dogs-64x2-CNN"

In [3]:
import pickle
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

In [4]:
#gpu_options = tg.GPUOptions(per_process_gpu_memory_fraction=0.333)
#sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

## Tensorboard for visualization

In [7]:
# Build the model
model = Sequential()

# Layers
model.add(Conv2D(64,(3,3),input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # to convert 3D data to 1D

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

#tensorboard
tensorboard = TensorBoard(log_dir=f"logs/{NAME}")

# compile the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

#train the model
model.fit(X,y,batch_size=32,epochs = 3,validation_split=0.3,callbacks=[tensorboard])

Train on 17462 samples, validate on 7484 samples
Epoch 1/3
17462/17462 [==============================] - 38s 2ms/step - loss: 0.6297 - acc: 0.6341 - val_loss: 0.6176 - val_acc: 0.6598
Epoch 2/3
17462/17462 [==============================] - 37s 2ms/step - loss: 0.5095 - acc: 0.7536 - val_loss: 0.4922 - val_acc: 0.7620
Epoch 3/3
17462/17462 [==============================] - 36s 2ms/step - loss: 0.4455 - acc: 0.7907 - val_loss: 0.4803 - val_acc: 0.7704


In [13]:
# save the trained model
model.save('64x3-CNN.model')

## Load the trained model and use it for prediction

In [18]:
import cv2
CATEGORIES = ["Dog", "Cat"]

def prepare(filepath):
    IMG_SIZE = 100  
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  

In [16]:
# load the model
model = tf.keras.models.load_model("64x3-CNN.model")

In [20]:
#pridiction on out-sample
prediction = model.predict([prepare('dog.jpg')])  # pass list to be predicted
prediction

array([[0.]], dtype=float32)

In [21]:
prediction[0][0]

0.0

In [22]:
CATEGORIES[int(prediction[0][0])]

'Dog'

In [25]:
prediction = model.predict([prepare('cat.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

[[1.]]
Cat
